In [2]:
#toxiguang
import os
import sys
#sys.path.insert(0, os.path.abspath('./.local/lib/python3.8/site-packages/'))
#sys.path.insert(0, os.path.abspath('/ihome/lwang/liw30/.local/bin'))

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
#from functions import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
import os
from PIL import Image, ImageSequence
from torch.utils import data
from tqdm import tqdm
import pandas as pd
import random
from sklearn.metrics import recall_score

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:4096"

In [4]:

print('3dcnn')
class Dataset_3DCNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, data,  frames, bSampling, minD, maxD, offset):
        "Initialization"
        self.data_path = data_path
        #self.datafile = datafile
        self.frames=31
        #filename=data_path+datafile
        #print(filename)
        data1=data
        
        labels=[]

        index=0

        count=int(data1.shape[0]/self.frames)
        print(count)
        page =np.ndarray(shape=(count, self.frames,20), dtype=float, order='F')
        page_d=np.ndarray(shape=(count, self.frames,20), dtype=float, order='F')
        index1=0
        index2=0
        sum1=0
        for index, row in data1.iterrows():
            index1=int(index/self.frames)
            index2=int(index % self.frames)
            if True: #skip%5==0:
  
                page[index1, index2,0]=row["nose_x"]
                page[index1, index2,1]=row["nose_y"]
                page[index1, index2,2]=row["objectA_x"]
                page[index1, index2,3]=row["objectA_y"]
                page[index1, index2,4]=row["left ear_x"]
                page[index1, index2,5]=row["left ear_y"]
                page[index1, index2,6]=row["right ear_x"]
                page[index1, index2,7]=row["right ear_y"]
                page[index1, index2,8]=row["neck_x"]
                page[index1, index2,9]=row["neck_y"]
                page[index1, index2,10]=row["middle back_x"]
                page[index1, index2,11]=row["middle back_y"]
                page[index1, index2,12]=row["middle left_x"]
                page[index1, index2,13]=row["middle left_y"]
                page[index1, index2,14]=row["middle right_x"]
                page[index1, index2,15]=row["middle right_y"]
                page[index1, index2,16]=row["tail buttom_x"]
                page[index1, index2,17]=row["tail buttom_y"]
                page[index1, index2,18]=row["tail mid_x"]
                page[index1, index2,19]=row["tail mid_y"]
                
                
            if index %self.frames==0:
                label=row['Label2']
                if label=='Random' :
                    labels.append(0)
                else:
                    labels.append(1) 
                    sum1=sum1+1
        page_d=torch.from_numpy(page)
        page_d=page_d.to(torch.float32)
        self.data1=page_d
        self.labels = labels      
        print('size xlabel')
        print(len(labels))
        print('sum of xlabels')
        print(sum1)

        #self.transform = transform
        #self.frames = 90
        #print(labels)
        #print(folders)
    def __len__(self):
        "Denotes the total number of samples"
        return len(self.labels)

    def __getitem__(self, index):
        "Generates one sample of data"
        # Select sample
       

        # Load data
        X = self.data1[index]#.unsqueeze_(0)     # (input) spatial images
        y = torch.LongTensor([self.labels[index]])                  # (labels) LongTensor are for int64 instead of FloatTensor

        # print(X.shape)
        return X, y
## ---------------------- end of Dataloaders ---------------------- ##
print('LSTMClassifier')

3dcnn
LSTMClassifier


In [5]:
            
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]
print('train')  
def train(log_interval, model, device, train_loader, optimizer, epoch):
    # set model as training mode
    model.train()
   
    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device


        X, y = X.to(device), y.to(device).view(-1, )

        N_count += X.size(0)

        optimizer.zero_grad()
        output = model(X)  # output size = (batch, number of classes)

        loss = F.cross_entropy(output, y)
        
        #l1_lambda = 0.01
        #l1_norm = sum(torch.linalg.norm(p, 1) for p in model.parameters())

        #loss = loss + l1_lambda * l1_norm
        
        
        losses.append(loss.item())

        # to compute accuracy
        y_pred = torch.max(output, 1)[1]  # y_pred != output
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()

        # show information
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))

    return losses, scores

print('validation')
def validation(model, device, optimizer, test_loader):
    # set model as testing mode
    model.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device).view(-1, )

            output = model(X)

            loss = F.cross_entropy(output, y, reduction='sum')
            test_loss += loss.item()                 # sum up batch loss
            y_pred = output.max(1, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability

            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(test_loader.dataset)

    # to compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())
    ylabel = all_y.cpu().data.squeeze().numpy()
    ypredict = all_y_pred.cpu().data.squeeze().numpy()
    sensi_score = recall_score(ylabel,ypredict)
    false_posi = recall_score(np.logical_not(ylabel),ypredict)
    false_negat = recall_score(ylabel,np.logical_not(ypredict))
    speci_score = recall_score(np.logical_not(ylabel),np.logical_not(ypredict))
    
    
    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))
    print("True_positive: " + str(round(sensi_score*100,4)) + "  False_positive: " + str(round(false_posi*100,4)))
    print("False_negative: " + str(round(false_negat*100,4)) + "  True_negative: " + str(round(speci_score*100,4)))
    # save Pytorch models of best record
    torch.save(model.state_dict(), os.path.join(save_model_path, '3dcnn_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, '3dcnn_optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    print("Epoch {} model saved!".format(epoch + 1))

    return test_loss, test_score, ylabel, ypredict


# Detect devices
print('detect device')
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU
print('params')
# load UCF101 actions names
batch_size=100
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 0, 'pin_memory': True} if use_cuda else {}
params1 = {'batch_size': batch_size, 'shuffle': False, 'num_workers': 0, 'pin_memory': True} if use_cuda else {}





#test_set= Dataset_3DCNN(data_path1, test_list,60, bSampling=False, minD=30, maxD=80,offset=0)

#test_loader = data.DataLoader(test_set, **params1) 

train
validation
detect device
params


In [6]:
# start training


In [7]:


data_path="D:/Xiguang/sample_data/DLC_data/LSTMdataset/"
save_model_path = "D:/Xiguang/sample_data/DLC_data/"
save_result_path = "D:/Xiguang/sample_data/DLC_data/"
train_data = pd.read_csv('D:/Xiguang/sample_data/DLC_data/LSTMdataset/train_set_combine-4-3_31fr.csv', index_col=False)
valid_data = pd.read_csv('D:/Xiguang/sample_data/DLC_data/LSTMdataset/test_set_combine-4-3_31fr.csv', index_col=False)


input_dim = 20 
hidden_dim = 256
layer_dim = 8
output_dim = 2
seq_dim = 60


n_epochs = 100
learning_rate = 1e-4
log_interval = 10 
batch_size=2000
train_set, valid_set = Dataset_3DCNN(data_path, train_data,31, bSampling=False, minD=30, maxD=80,offset=0), \
                       Dataset_3DCNN(data_path, valid_data,31, bSampling=False,minD=30, maxD=80,offset=0 )
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 0, 'pin_memory': True} if use_cuda else {}
params2 = {'batch_size': batch_size, 'shuffle': False, 'num_workers': 0, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(train_set, **params)
valid_loader = data.DataLoader(valid_set, **params2) 
iterations_per_epoch = len(train_loader)

learning_rate = 1e-4
best_acc = 0
patience, trials = 100, 0
print('before model')
model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
model = model.cuda()
print('before opt')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 0)   # optimize all cnn parameters

use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

# load UCF101 actions names


# record training process
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []


for epoch in range(n_epochs):
    # train, test model

    train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)

    epoch_test_loss, epoch_test_score, ylabel1, ypredict1 = validation(model, device, optimizer, valid_loader)

    #epoch_test_loss1, epoch_test_score1, ylabel2, ypredict2 = validation(model, device, optimizer, test_loader)
    
    A = np.array(ylabel1)
        
    B = np.array(ypredict1)
    
    # prediction save part
    
    np.savetxt(save_result_path+'y_label_prediction1_'+str(epoch)+'.txt',  np.c_[A, B],fmt='%i', delimiter=',')
        

   


25636
size xlabel
25636
sum of xlabels
13050
6467
size xlabel
6467
sum of xlabels
3306
before model
before opt
Train Epoch: 1 [20000/25636 (77%)]	Loss: 0.693166, Accu: 50.15%

Test set (6467 samples): Average loss: 0.6922, Accuracy: 51.12%

True_positive: 100.0  False_positive: 100.0
False_negative: 0.0  True_negative: 0.0
Epoch 1 model saved!
Train Epoch: 2 [20000/25636 (77%)]	Loss: 0.683204, Accu: 57.15%

Test set (6467 samples): Average loss: 0.6685, Accuracy: 62.83%

True_positive: 71.6878  False_positive: 46.441
False_negative: 28.3122  True_negative: 53.559
Epoch 2 model saved!
Train Epoch: 3 [20000/25636 (77%)]	Loss: 0.648748, Accu: 64.20%

Test set (6467 samples): Average loss: 0.6574, Accuracy: 62.30%

True_positive: 58.0762  False_positive: 33.2806
False_negative: 41.9238  True_negative: 66.7194
Epoch 3 model saved!


KeyboardInterrupt: 